In [1]:
####################################
#ENVIRONMENT SETUP

In [2]:
#Importing Libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr

import sys; import os; import time; from datetime import timedelta
import pickle
import h5py

In [3]:
#MAIN DIRECTORIES
def GetDirectories():
    mainDirectory='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
    mainCodeDirectory=os.path.join(mainDirectory,"Code/CodeFiles/")
    scratchDirectory='/mnt/lustre/koa/scratch/air673/'
    codeDirectory=os.getcwd()
    return mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory

[mainDirectory,mainCodeDirectory,scratchDirectory,codeDirectory] = GetDirectories()

In [4]:
#IMPORT CLASSES (from current directory)
sys.path.append(os.path.join(mainCodeDirectory,"2_Variable_Calculation"))
from CLASSES_Variable_Calculation import ModelData_Class, SlurmJobArray_Class, DataManager_Class

In [5]:
import sys
path=os.path.join(mainCodeDirectory,'Functions/')
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [6]:
####################################
#LOADING CLASSES

In [7]:
#data loading class
ModelData = ModelData_Class(mainDirectory, scratchDirectory, simulationNumber=1)
#data manager class
DataManager = DataManager_Class(mainDirectory, scratchDirectory, ModelData.res, ModelData.t_res, ModelData.Nz_str,
                                ModelData.Np_str, dataType="CalculateMoreVariables", dataName="MoistureConvergence",
                                dtype='float32')

=== CM1 Data Summary ===
 Simulation #:   1
 Resolution:     1km
 Time step:      5min
 Vertical levels:34
 Parcels:        1e6
 Data file:      /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_1km_5min_34nz.nc
 Parcel file:    /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Model/cm1r20.3/run/cm1out_pdata_1km_5min_1e6np.nc
 Time steps:     133

=== DataManager Summary ===
 inputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData
 outputDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables
 inputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/TimeSplitModelData/1km_5min_34nz/ModelData
 outputDataDirectory #:   /mnt/lustre/koa/koastore/torri_group/air_directory/Projec

In [8]:
#JOB ARRAY SETUP
UsingJobArray=True

def GetNumJobs(res):
    if res=='1km':
        num_jobs=20
    elif res=='250m': 
        num_jobs=100
    return num_jobs
num_jobs = GetNumJobs(ModelData.res)
SlurmJobArray = SlurmJobArray_Class(total_elements=ModelData.Ntime, num_jobs=num_jobs, UsingJobArray=UsingJobArray)
start_job = SlurmJobArray.start_job; end_job = SlurmJobArray.end_job

def GetNumElements():
    num_elements = np.arange(ModelData.Ntime)[start_job:end_job]
    return num_elements
num_elements = GetNumElements()

Running timesteps from 0:7 



In [9]:
####################################
#FUNCTIONS

In [10]:
#FROM MOISTURE FLUX CONVERGENCE: ITS HISTORY AND APPLICATION IN CONVECTIVE INITIATION FORECASTING
# Peter C. Banacos
# NOAA/NWS/NCEP/Storm Prediction Center
# Norman, Oklahoma 73069
# David M. Schultz
# Cooperative Institute for Mesoscale Meteorological Studies, Univ. of Oklahoma, and
# NOAA/National Severe Storms Laboratory, Norman, Oklahoma 73069

In [11]:
#Calculation Functions
def GetInputVariables(inputDataDirectory, timeString):
    u_data = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="uinterp")
    v_data = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="vinterp") 
    qv_data = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="qv") 
    rho_data = DataManager.GetTimestepData(inputDataDirectory, timeString, variableName="rho") 

    InputDictionary = {'u_data': u_data,
                       'v_data': v_data,
                       'qv_data': qv_data,
                       'rho_data': rho_data}
    return InputDictionary

def VariableCalculation(inputDictionary):
    [u_data, v_data, qv_data, rho_data] = (inputDictionary[k] for k in ['u_data', 'v_data', 'qv_data', 'rho_data'])

    if ModelData.res=='1km':
        dx=1000; dy=1000
    elif ModelDatares=='250m':
        dx=250; dy=250

    #calculating moisture convergence
    #NOTE: model was set to output interpolated (scalar) version of U and V
    #NOTE: so I used central difference on the scalar versions of U and V
    one=Ddx_3D(rho_data*qv_data*u_data,dx)
    two=Ddy_3D(rho_data*qv_data*v_data,dy)
    HMC=-(one+two)

    outputDictionary = {
        'HMC': HMC
    }
    return outputDictionary

In [12]:
####################################
#RUNNING

In [13]:
#CALCULATING AND APPENDING TO DATA EACH TIMESTEP
for t in num_elements:
    if np.mod(t,1)==0: print(f'Current time {t}')

    #getting timestring for loading input data
    timeString = ModelData.timeStrings[t]

    #loading input variables
    inputDictionary = GetInputVariables(DataManager.inputDataDirectory, timeString)

    #calculating variables
    outputDictionary = VariableCalculation(inputDictionary)
    
    #outputting
    DataManager.SaveOutputTimestep(DataManager.outputDataDirectory, timeString, outputDictionary)

Current time 0
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/MoistureConvergence/MoistureConvergence_1km_5min_34nz_0-00-00.h5
Current time 1
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/MoistureConvergence/MoistureConvergence_1km_5min_34nz_0-05-00.h5
Current time 2
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/MoistureConvergence/MoistureConvergence_1km_5min_34nz_0-10-00.h5
Current time 3
Saved timestep to output file: /mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/Code/OUTPUT/Variable_Calculation/CalculateMoreVariables/1km_5min_34nz/MoistureConvergence/MoistureConvergence_1km_5min_34nz

In [ ]:
######################################################

In [15]:
# #READING BACK IN
# def GetVariableData(t, dataName="MoistureConvergence"):
#     res = ModelData.res
#     t_res = ModelData.t_res
#     Nz_str = ModelData.Nz_str
#     inputDirectory = os.path.join(DataManager.outputDirectory, f"{res}_{t_res}_{Nz_str}nz", dataName)
#     timeString = ModelData.timeStrings[t]

#     FileName = os.path.join(inputDirectory, f"{dataName}_{res}_{t_res}_{Nz_str}nz_{timeString}.h5")

#     dataDictionary = {}
#     with h5py.File(FileName, 'r') as f:
#         print("Keys in file:", list(f.keys()))
#         for key in f.keys():
#             dataDictionary[key] = f[key][:]
#             print(f"{key}: shape = {dataDictionary[key].shape}, dtype = {dataDictionary[key].dtype}")
#     return dataDictionary

# dataDictionary = GetVariableData(t=0)